## CLASSICAL AUTOENCODERS

In [1]:
from keras.layers import Input, Dense
from keras.models import Model

input_img = Input(shape=(784,))
encoded = Dense(32, activation='relu')(input_img)  
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# get the encoder and decoder as seperate models
# encoder
encoder = Model(input_img, encoded)

# decoder
encoded_input = Input(shape=(32,))  # encoding_dim = 32
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [7]:
from keras.datasets import mnist
import numpy as np
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [8]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                validation_data=(x_test, x_test),
                verbose=1)

Epoch 1/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6836 - val_loss: 0.6833
Epoch 2/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6833 - val_loss: 0.6830
Epoch 3/50
235/235 [==============================] - 1s 5ms/step - loss: 0.6830 - val_loss: 0.6827
Epoch 4/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6827 - val_loss: 0.6824
Epoch 5/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6824 - val_loss: 0.6820
Epoch 6/50
235/235 [==============================] - 1s 5ms/step - loss: 0.6820 - val_loss: 0.6817
Epoch 7/50
235/235 [==============================] - 1s 4ms/step - loss: 0.6817 - val_loss: 0.6813
Epoch 8/50
235/235 [==============================] - 1s 5ms/step - loss: 0.6813 - val_loss: 0.6809
Epoch 9/50
235/235 [==============================] - 2s 7ms/step - loss: 0.6810 - val_loss: 0.6805
Epoch 10/50
235/235 [==============================] - 2s 7ms/step - loss: 0.6806 - val_loss: 0.6802

In [9]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

313/313 [==============================] - 0s 964us/step


## AUTOENCODERS CUM NEURAL NETWORKS (AUTOENCODERS ARE USED AS PREPROCESSORS)

In [17]:
from tensorflow.keras.utils import to_categorical

num_classes = 10  # The number of classes in the MNIST dataset

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [20]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Load the MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()

# Normalize pixel values to between 0 and 1
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Flatten the images
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Define the dimensions for the encoding and hidden layers of the autoencoder
encoding_dim = 32  

# Create an autoencoder model
input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)

# Create a separate model for encoding
encoder = Model(input_img, encoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
###autoencoder.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test, x_test))
autoencoder.fit(x_train, x_train, epochs=15, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

# Use the encoder to transform the data
encoded_x_train = encoder.predict(x_train)
encoded_x_test = encoder.predict(x_test)

# Define a neural network for classification
model = Sequential()
#model.add(Dense(64, activation='relu', input_dim=encoding_dim))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the neural network on the encoded data
model.fit(encoded_x_train, y_train, epochs=10, batch_size=128, validation_data=(encoded_x_test, y_test))


Epoch 1/15
235/235 [==============================] - 2s 5ms/step - loss: 0.0713 - val_loss: 0.0418
Epoch 2/15
235/235 [==============================] - 1s 4ms/step - loss: 0.0355 - val_loss: 0.0294
Epoch 3/15
235/235 [==============================] - 1s 5ms/step - loss: 0.0259 - val_loss: 0.0224
Epoch 4/15
235/235 [==============================] - 1s 5ms/step - loss: 0.0209 - val_loss: 0.0187
Epoch 5/15
235/235 [==============================] - 1s 5ms/step - loss: 0.0178 - val_loss: 0.0162
Epoch 6/15
235/235 [==============================] - 1s 5ms/step - loss: 0.0156 - val_loss: 0.0143
Epoch 7/15
235/235 [==============================] - 1s 5ms/step - loss: 0.0140 - val_loss: 0.0130
Epoch 8/15
235/235 [==============================] - 1s 5ms/step - loss: 0.0128 - val_loss: 0.0120
Epoch 9/15
235/235 [==============================] - 1s 5ms/step - loss: 0.0120 - val_loss: 0.0114
Epoch 10/15
235/235 [==============================] - 1s 5ms/step - loss: 0.0115 - val_loss: 0.0110